In [ ]:
## from xml.dom import minidom

all_ontologies = list(set([item + ".owl" for sublist in [f.split(".")[0].split("-") for  f in os.listdir(alignment_folder)] for item in sublist]))
ontologies_folder = "conference_ontologies/"

def parse_classes(ontology):
    prefix = "http://" + ontology.split(".")[0] + "#"
    doc = minidom.parse(ontology)
    curr_classes = [el.getAttribute("rdf:ID") for el in doc.getElementsByTagName('owl:Class')]
    classes = [prefix + el for el in curr_classes if el]
    return classes

def parse_properties(ontology):
    properties = []
    for onto in ontologies:
        prefix = "http://" + onto.split(".")[0] + "#"
        doc = minidom.parse(folder + onto)
        root = doc.documentElement
        curr_properties = [el.getAttribute("rdf:ID") for el in doc.getElementsByTagName('ObjectProperty')]
        properties.extend([prefix + el for el in curr_properties if el])
    return properties

classes, properties = [], []
for ontology in all_ontologies:
    classes.extend(parse_classes(ontologies_folder + ontology))
    properties.extend(parse_properties(ontologies_folder + ontology))
properties = load_properties(ontologies_folder)


In [ ]:
def replace_hyphen(ontology, replaced_props):
    onto_str = open(ontology).read()
    for (a,b) in replaced_props:
        onto_str = onto_str.replace(a,b)
    ontology_temp = ontology.split(".")[0] + "_temp.owl"
    open(ontology_temp, "w+").write(onto_str)
    return ontology_temp

replaced_props = [(str(prop).split(".")[1], str(prop).split(".")[1].replace("-", "_")) for prop in list(onto.properties()) if "-" in str(prop)]
replaced_ontology = replace_hyphen(ontology, replaced_props)
replaced_onto = get_ontology(replaced_ontology).load()
# os.remove(replaced_ontology)

In [ ]:
prefix = "http://" + ontology.split(".")[0] + "#"
doc = minidom.parse(ontology)
root = doc.documentElement
curr_properties = [el.getAttribute("rdf:ID") for el in doc.getElementsByTagName('Class')]
# properties.extend([prefix + el for el in curr_properties if el])

In [ ]:
props = [el for el in root.childNodes if type(el) == minidom.Element and el._get_tagName() == 'owl:ObjectProperty'] 
prop_ids = [prop.getAttribute('rdf:ID') for prop in props if prop.getAttribute('rdf:ID')]

In [ ]:
doc = minidom.parse(ontology)
object_props = [el.getAttribute("rdf:ID") for el in doc.getElementsByTagName('owl:ObjectProperty')]
filt = [[e for e in el._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == "rdf:type"] for el in doc.getElementsByTagName('owl:FunctionalProperty')]
[el[0].getAttribute("rdf:resource") for el in filt if el]


In [ ]:
# linear search with USE, no greedy

opt_threshold, optimum_metrics = -1000, [-1000 for i in range(3)]
t = time.time()
for j,threshold in enumerate(np.arange(0.15, 1.0005, 0.01)):
    print ("threshold =", threshold, "Time = ", time.time()-t) 
    pred = ["F" for key in train_data]
    mapped = []
    for i,key in enumerate(train_data):
        if train_data[key][0] > threshold:
            pred[i] = "T"
        mapped.extend(list(key))
    gt = [l[1] for l in train_data.values()]
    f1score = f1_score(gt, pred, pos_label="T")
    if f1score > optimum_metrics[-1]:
        optimum_metrics = [precision_score(gt, pred, pos_label="T"), recall_score(gt, pred, pos_label="T"), f1score]
        opt_threshold = threshold
    

In [ ]:
threshold = opt_threshold
pred = ["F" for key in test_data]
mapped = []
for i,key in enumerate(test_data):
    if test_data[key][0] > threshold:
        pred[i] = "T"
    mapped.extend(list(key))
gt = [l[1] for l in test_data.values()]
f1score = f1_score(gt, pred, pos_label="T")
metrics = [precision_score(gt, pred, pos_label="T"), recall_score(gt, pred, pos_label="T"), f1score]

In [ ]:
# Data Generation for RotatE embeddings from OpenKE

ontologies_in_alignment = [l.split(".")[0].split("-") for l in os.listdir("reference-alignment/")]

def generate(ontology, concept_prefix, id_prefix=[0,0]):
    ont = Ontology(ontology)
    classes_dict = {concept_prefix+"#"+elem : str(id_prefix[0] + i) for i,elem in enumerate(ont.get_classes())} 
    props = ont.get_object_properties() + ont.get_data_properties() + ["subclass_of"]
    prop_dict = {concept_prefix+"#"+elem : str(id_prefix[1] + i) for i,elem in enumerate(props)} 
    triplets = ["\t".join((classes_dict[concept_prefix+"#"+el[0]], classes_dict[concept_prefix+"#"+el[1]], prop_dict[concept_prefix+"#"+el[2]]))
                for el in ont.get_triples()]
    return classes_dict, prop_dict, triplets

    
for l in ontologies_in_alignment:
    ont1 = "conference_ontologies/" + l[0] + ".owl"
    ont2 = "conference_ontologies/" + l[1] + ".owl"
    
    benchmark_dir = "OpenKE/benchmarks/" + l[0] + "-" + l[1]
    if not os.path.isdir(benchmark_dir):
        os.mkdir(benchmark_dir)
    
    c1, p1, t1 = generate(ont1, l[0])
    c2, p2, t2 = generate(ont2, l[1], [len(c1), len(p1)])
    
    classes_dict = {**c1, **c2}
    prop_dict = {**p1, **p2}
    triplets = t1 + t2
    
    entity_str = str(len(classes_dict)) + "\n" + "\n".join(["\t".join(elem) for elem in classes_dict.items()])
    prop_str = str(len(prop_dict)) + "\n" + "\n".join(["\t".join(elem) for elem in prop_dict.items()])
    triplets_str = str(len(triplets)) + "\n" + "\n".join(triplets)

    open(benchmark_dir + "/entity2id.txt", "w+").write(entity_str)
    open(benchmark_dir + "/relation2id.txt", "w+").write(prop_str)
    open(benchmark_dir + "/train2id.txt", "w+").write(triplets_str)

In [ ]:
# AML test
results = []
for i in list(range(0, len(all_ont_pairs), 3)):
    test_onto = all_ont_pairs[i:i+3]
    for ont_pair in test_onto:
        a, b, c = ont_pair[0], ont_pair[1], ont_pair[0] + "-" + ont_pair[1]
        java_command = "java -jar AML_v3.1/AgreementMakerLight.jar -s conference_ontologies/" + a + ".owl" + \
                            " -t conference_ontologies/" + b + ".owl -o AML-test-results/" + c + ".rdf -a"
    #     print (java_command)
        process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    #     print (a,b,c)

    pred_aml = load_alignments("AML-test-results/")
    pred_aml = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml]
    tp = len([elem for elem in pred_aml if data[elem][1] == "T"])
    fn = len([key for key in gt_mappings if key not in set(pred_aml) and not is_valid(test_onto, key)])
    fp = len([elem for elem in pred_aml if data[elem][1] == "F"])

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = 2 * precision * recall / (precision + recall)
    f2score = 5 * precision * recall / (4 * precision + recall)
    f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    print (precision, recall, f1score, f2score, f0_5score)
    
    metrics = [precision, recall, f1score, f2score, f0_5score]
    results.append(metrics)
    
    rm_command = "rm -rf AML-test-results/*"
    process = subprocess.Popen(rm_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    
print ("Final Results:", np.mean(results, axis=0))

In [ ]:

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)


relations = []
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    triple_pairs = [list(itertools.product(el[0].split("###"), el[1].split("###"))) for el in ont.get_triples()]
    triple_pairs = [tuple([ont_name + "#" + concept for concept in triple]) for triple in flatten(triple_pairs)]
    relations.extend(triple_pairs)

relations = list(set(relations))
entities = list(set(flatten(relations)))

emb_indexer = {word: i for i, word in enumerate(list(embeddings.keys()))}
emb_vals = list(embeddings.values())

def generate_data(onto):
#     print (onto)
    relations_sub = [elem for elem in relations if elem[0].split("#")[0] == onto]
    entities_sub = [elem for elem in entities if elem.split("#")[0] == onto]
    entity_idx = {elem: (i, emb_indexer[elem.split("#")[1]]) for (i,elem) in enumerate(entities_sub) }
    
    edges = np.array([(entity_idx[a][0], entity_idx[b][0]) for (a,b) in relations_sub])
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), 
                        shape=(len(entities_sub), len(entities_sub)), dtype=np.float32)
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))
    adj = np.array(adj.todense())
    return (entity_idx, adj)

def generate_input_tensors(elem_tuple):
    data = []
    for elem in elem_tuple:
        entity_idx, adj = generate_data(elem.split("#")[0])
        emb_idx = torch.LongTensor([[entity_idx[elem][1]]])
        adj = torch.FloatTensor(adj)
        data.append((emb_idx, adj))
    return data
        
data_ontologies = {ont: generate_data(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
entities_set = set(entities)
nn_data_true = [key for key in data if data[key][-1] == "T" and key[0] in entities_set and key[1] in entities_set]
nn_data_false = [key for key in data if data[key][-1] == "F" and key[0] in entities_set and key[1] in entities_set]


lr = 0.001
num_epochs = 10
weight_decay = 0.0001
batch_size = 10
batch_size = min(batch_size, len(nn_data_true))
num_batches = int(ceil(len(nn_data_true)/batch_size))

model = SiameseNetwork()

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

for epoch in range(num_epochs):
    np.random.shuffle(nn_data_true)
    np.random.shuffle(nn_data_false)
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx+1) * batch_size
            
        pos_elems = np.array(nn_data_true)[batch_start:batch_end]
        neg_elems = np.array(nn_data_false)[batch_start:batch_end]
        optimizer.zero_grad()

        pos_data = [(torch.LongTensor([1]), generate_input_tensors(pos_elem)) for pos_elem in pos_elems]
        neg_data = [(torch.LongTensor([0]), generate_input_tensors(neg_elem)) for neg_elem in neg_elems]
        
        dat = pos_data + neg_data
        np.random.shuffle(dat)
        targets, inputs = list(zip(*dat))
        np.array(inputs).reshape
        outputs = model(inputs)

        loss = F.cross_entropy(outputs, targets)
        loss.backward()
        
        optimizer.step()
        
        if i%10 == 0:
            print ("Epoch: {} Idx: {} Loss: {}".format(epoch, i, loss.item()))

In [ ]:
# import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, inp, adj):
        print (inp.shape, self.W.shape)
        h = torch.mm(inp, self.W)
        N = h.size()[0] 

        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, h)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


class SiameseNetwork(nn.Module):
    def __init__(self, nfeat=512, nhid=100, dropout=0.3, alpha=0.001, nheads=8):
        super().__init__()
        
        self.name_embedding = nn.Embedding(len(embeddings), 512)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
        
        self.dropout = dropout

        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = GraphAttentionLayer(nhid * nheads, 2, dropout=dropout, alpha=alpha, concat=False)

    def forward(self, emb_indices, adj_mats):
        results = []
        for i in range(2):
            x = torch.flatten(self.name_embedding(emb_indices))
            print (x.shape)
            adj = data[i][1]
            x = F.dropout(x, self.dropout, training=self.training)
            x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
            x = F.leaky_relu(F.dropout(x, self.dropout, training=self.training))
            results.append(x)
        x = torch.abs(results[1] - results[0])
        x = F.elu(self.out_att(x, adj))
        return F.log_softmax(x, dim=1)




In [ ]:

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

nn_data_true = [key for key in data if data[key][-1] == "T" and key[0] in entities_set and key[1] in entities_set]
nn_data_false = [key for key in data if data[key][-1] == "F" and key[0] in entities_set and key[1] in entities_set]

In [ ]:

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)


relations = []
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    triple_pairs = [list(itertools.product(el[0].split("###"), el[1].split("###"))) for el in ont.get_triples()]
    triple_pairs = [tuple([ont_name + "#" + concept for concept in triple]) for triple in flatten(triple_pairs)]
    relations.extend(triple_pairs)

relations = list(set(relations))
entities = list(set(flatten(relations)))

emb_indexer = {word: i for i, word in enumerate(list(embeddings.keys()))}
emb_vals = list(embeddings.values())

def generate_data(onto):
#     print (onto)
    relations_sub = [elem for elem in relations if elem[0].split("#")[0] == onto]
    entities_sub = [elem for elem in entities if elem.split("#")[0] == onto]
    entity_idx = {elem: (i, emb_indexer[elem.split("#")[1]]) for (i,elem) in enumerate(entities_sub) }
    
    edges = np.array([(entity_idx[a][0], entity_idx[b][0]) for (a,b) in relations_sub])
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), 
                        shape=(len(entities_sub), len(entities_sub)), dtype=np.float32)
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))
    adj = np.array(adj.todense())
    return (entity_idx, adj)

def generate_input_tensors(elem_tuple):
    data = []
    for elem in elem_tuple:
        entity_idx, adj = generate_data(elem.split("#")[0])
        emb_idx = entity_idx[elem][1]
        data.append((emb_idx, adj))
    return data
        
data_ontologies = {ont: generate_data(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
entities_set = set(entities)
nn_data_true = [key for key in data if data[key][-1] == "T" and key[0] in entities_set and key[1] in entities_set]
nn_data_false = [key for key in data if data[key][-1] == "F" and key[0] in entities_set and key[1] in entities_set]

torch.set_default_dtype(torch.float64)

lr = 0.001
num_epochs = 10
weight_decay = 0.0001
batch_size = 10
batch_size = min(batch_size, len(nn_data_true))
num_batches = int(ceil(len(nn_data_true)/batch_size))

model = SiameseNetwork()

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

for epoch in range(num_epochs):
    np.random.shuffle(nn_data_true)
    np.random.shuffle(nn_data_false)
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx+1) * batch_size
            
        pos_elems = np.array(nn_data_true)[batch_start:batch_end]
        neg_elems = np.array(nn_data_false)[batch_start:batch_end]
        optimizer.zero_grad()

        pos_data = [(torch.LongTensor([1]), generate_input_tensors(pos_elem)) for pos_elem in pos_elems]
        neg_data = [(torch.LongTensor([0]), generate_input_tensors(neg_elem)) for neg_elem in neg_elems]
        
        dat = pos_data + neg_data
        np.random.shuffle(dat)
        targets, inputs = list(zip(*dat))
        emb_indices, adj_mats = np.array(inputs).reshape(2,-1)
        outputs = model(inputs)

        loss = F.cross_entropy(outputs, targets)
        loss.backward()
        
        optimizer.step()
        
        if i%10 == 0:
            print ("Epoch: {} Idx: {} Loss: {}".format(epoch, i, loss.item()))

In [ ]:
emb_indexer = {word: i for i, word in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: word for i, word in enumerate(list(embeddings.keys()))}
emb_vals = list(embeddings.values())

def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

def generate_data(elem_tuple):
    return np.array([emb_indexer[elem] for elem in elem_tuple])

def generate_input(elems, target):
    inputs = np.array([generate_data(elem) for elem in list(elems)])
    targets = np.array([target for i in range(len(elems))])

    indices = np.random.permutation(inputs.shape[0])
    inputs, targets = inputs[indices], targets[indices]
    inputs = torch.LongTensor(list(zip(*inputs)))
    targets = torch.LongTensor(targets)
    
    return inputs, targets
    
data = OrderedDict(sorted(data.items(),  key=lambda x:x[1][0], reverse=True))
all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))
all_results = []
for i in list(range(0, len(all_ont_pairs), 3)):
    test_onto = all_ont_pairs[i:i+3]
    
    train_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) not in test_onto}
    test_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) in test_onto}

    torch.set_default_dtype(torch.float64)
    
    train_test_split = 0.9

    train_data_t = [key for key in train_data if data[key][-1] == "T"]
    train_data_f = [key for key in train_data if data[key][-1] == "F"]
    
    lr = 0.001
    num_epochs = 50
    weight_decay = 0.001
    batch_size = 10
    dropout = 0.3
    batch_size = min(batch_size, len(train_data_t))
    num_batches = int(ceil(len(train_data_t)/batch_size))
    
    model = SiameseNetwork()

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(num_epochs):
        np.random.shuffle(train_data_t)
        np.random.shuffle(train_data_f)
        train_data_f = train_data_f[:len(train_data_t)]
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            pos_elems = np.array(train_data_t)[batch_start:batch_end]
            neg_elems = np.array(train_data_f)[batch_start:batch_end]
            optimizer.zero_grad()
            
            inputs_pos, targets_pos = generate_input(pos_elems, 1)
            inputs_neg, targets_neg = generate_input(neg_elems, 0)
            
            outputs_pos = model(inputs_pos)
            loss_pos = F.cross_entropy(outputs_pos, targets_pos)
            
            outputs_neg = model(inputs_neg)
            loss_neg = F.cross_entropy(outputs_neg, targets_neg)
            
            loss = loss_pos + loss_neg
            loss.backward()

            optimizer.step()

            if batch_idx%10 == 0:
                print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))

    model.eval()
    
    test_data_t = [key for key in test_data if data[key][-1] == "T"]
    test_data_f = [key for key in test_data if data[key][-1] == "F"]
    
    with torch.no_grad():
        all_pred = []
        batch_size = min(batch_size, len(test_data_t))
        num_batches = int(ceil(len(test_data_t)/batch_size))

        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)
        test_data_f = test_data_f[:len(test_data_t)]

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            pos_elems = np.array(test_data_t)[batch_start:batch_end]
            neg_elems = np.array(test_data_f)[batch_start:batch_end]
            optimizer.zero_grad()

            inputs = np.array([generate_data(elem) for elem in list(pos_elems) + list(neg_elems)])
            targets = np.array([1 for i in range(len(pos_elems))] + [0 for i in range(len(neg_elems))])

            indices = np.random.permutation(inputs.shape[0])
            inputs, targets = inputs[indices], targets[indices]
            inputs = torch.LongTensor(list(zip(*inputs)))
            targets = torch.LongTensor(targets)

            outputs = model(inputs)

            _, predicted = torch.max(outputs, 1)
            predicted = [el.item() for el in predicted]
            
            targets = [el.item() for el in targets]

            for idx, pred_elem in enumerate(predicted):
                if pred_elem == 1:
                    ent1 = emb_indexer_inv[inputs.numpy()[0][idx]]
                    ent2 = emb_indexer_inv[inputs.numpy()[1][idx]]
                    all_pred.append((ent1, ent2))
        
        
        tp = len([elem for elem in all_pred if test_data[elem][1] == "T"])
        fn = len([key for key in gt_mappings if key not in set(all_pred) and is_valid(test_onto, key)])
        fp = len([elem for elem in all_pred if test_data[elem][1] == "F"])

        try:
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            f1score = 2 * precision * recall / (precision + recall)
            f2score = 5 * precision * recall / (4 * precision + recall)
            f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
        except Exception as e:
            print (e)
            continue
        
        print ("Precision: {} Recall: {} F1-Score: {} F2-Score: {} F0.5-Score: {}".format(
            precision, recall, f1score, f2score, f0_5score))
        all_results.append((precision, recall, f1score, f2score, f0_5score))

print ("Final Results: ", np.mean(all_results, axis=0))

In [ ]:
def test():
    global batch_size, test_data_t, test_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics
    all_results = []
    with torch.no_grad():
        all_pred = []
        batch_size = min(batch_size, len(test_data_t))
        num_batches = int(ceil(len(test_data_t)/batch_size))
        batch_size_f = int(ceil(len(test_data_f)/num_batches))
        
        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            batch_start_f = batch_idx * batch_size_f
            batch_end_f = (batch_idx+1) * batch_size_f
            
            pos_elems = np.array(test_data_t)[batch_start:batch_end]
            neg_elems = np.array(test_data_f)[batch_start_f:batch_end_f]
            optimizer.zero_grad()

            inputs = np.array([generate_data(elem) for elem in list(pos_elems) + list(neg_elems)])
            targets = np.array([1 for i in range(len(pos_elems))] + [0 for i in range(len(neg_elems))])

            indices = np.random.permutation(inputs.shape[0])
            inputs, targets = inputs[indices], targets[indices]
            inputs = torch.LongTensor(list(zip(*inputs)))
            targets = torch.DoubleTensor(targets)

            outputs = model(inputs)
            outputs /= torch.sum(outputs, dim=1).view(-1, 1)
#             outputs = 1 - outputs

            values, predicted = torch.max(outputs, 1)
            predicted = [el.item() for el in predicted]
            
            targets = [el.item() for el in targets]
            
            write (("Outputs:", [str(s) for s in list(outputs.numpy())]))
            write (("Targets:", [str(s) for s in targets]))
            for idx, pred_elem in enumerate(predicted):
                if pred_elem == 1:
                    ent1 = emb_indexer_inv[inputs.numpy()[0][idx]]
                    ent2 = emb_indexer_inv[inputs.numpy()[1][idx]]
                    all_pred.append((ent1, ent2))
        
        
        tp = len([elem for elem in all_pred if test_data[elem][1] == "T"])
        fn = len([key for key in gt_mappings if key not in set(all_pred) and not is_valid(test_onto, key)])
        fp = len([elem for elem in all_pred if test_data[elem][1] == "F"])

        try:
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            f1score = 2 * precision * recall / (precision + recall)
            f2score = 5 * precision * recall / (4 * precision + recall)
            f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
        except Exception as e:
            print (e)
            return
        
        print ("Precision: {} Recall: {} F1-Score: {} F2-Score: {} F0.5-Score: {}".format(
            precision, recall, f1score, f2score, f0_5score))
        all_results.append((precision, recall, f1score, f2score, f0_5score))

    print ("Final Results: ", np.mean(all_results, axis=0))

In [3]:
def get_misclassified(input_list, feat):
#     print (input_list, feat)
    return [el for el in input_list if features_dict[el[0].split("#")[1]]["type"] in feat]


#     for i,key in enumerate(all_results):
#         if all_results[key][0] > opt_threshold:
#             res.append(key)

#     fn_list = [key for key in gt_mappings if key not in set(res) and not is_valid(test_onto, key)]
#     fp_list = [elem for elem in res if all_results[elem][1] == "F"]
#     tp_list = [elem for elem in res if all_results[elem][1] == "T"]
            
            
#     fn_prop, fn_entity = get_misclassified(fn_list, ["property", "triple"]), get_misclassified(fn_list, ["entity"])
#     fp_prop, fp_entity = get_misclassified(fp_list, ["property", "triple"]), get_misclassified(fp_list, ["entity"])

#     write(("False negative props: ", "\n".join([str(s) for s in fn_prop]), "False negative entities: ", "\n".join([str(s) for s in fn_entity])))
#     write(("False positive props: ", "\n".join([str(s) for s in fp_prop]), "False positive entities: ", "\n".join([str(s) for s in fp_entity])))

#     write ("Fn prop: {} Fn entity: {} Prop misclassification Percentage: {}".format(len(fn_prop), len(fn_entity), float(len(fn_prop)/(len(fn_entity)+len(fn_prop)))))
    
#     tot_prop, tot_ent = get_misclassified(test_data, ["property", "triple"]), get_misclassified(test_data, ["entity"])
#     write("Total proportion of props: " + str(float(len(tot_prop)/(len(tot_ent)+len(tot_prop)))))

tensor([[ 0.0688, 23.0000],
        [ 1.0000,  4.0000]])

In [ ]:
# Naive Greedy

def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

data = OrderedDict(sorted(data.items(),  key=lambda x:x[1][0], reverse=True))
all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))
results = []
failed = []
for i in list(range(0, len(all_ont_pairs), 3)):
    test_onto = all_ont_pairs[i:i+3]
    
    train_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) not in test_onto}
    test_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) in test_onto}

    opt_threshold, optimum_metrics = -1000, [-1000 for i in range(5)]
    t = time.time()
    for j,threshold in enumerate(np.arange(0.15, 1.0005, 0.01)):
        print ("threshold =", threshold, "Time = ", time.time()-t) 
        pred = []
        for i,key in enumerate(train_data):
            if train_data[key][0] > threshold:
                pred.append(key)

        tp = len([elem for elem in pred if train_data[elem][1] == "T"])
        fn = len([key for key in gt_mappings if key not in set(pred) and is_valid(test_onto, key)])
        fp = len([elem for elem in pred if train_data[elem][1] == "F"])
        precision, recall, f1score, f2score, f0_5score = [None for i in range(5)] 
        try:
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            f1score = 2 * precision * recall / (precision + recall)
            f2score = 5 * precision * recall / (4 * precision + recall)
            f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
        except Exception as e:
            print (e)
            continue
        print (precision, recall, f1score, f2score, f0_5score)

        if f1score > optimum_metrics[2]:
            optimum_metrics = [precision, recall, f1score, f2score, f0_5score]
            opt_threshold = threshold
    
    threshold = opt_threshold
    pred = []
    for i,key in enumerate(test_data):
        if test_data[key][0] > threshold:
            pred.append(key)

    curr = dict()
    
    curr["fn"] = [key for key in gt_mappings if key not in set(pred) and not is_valid(test_onto, key)]
    curr["fp"] = [elem for elem in pred if test_data[elem][1] == "F"]
    tp = len([elem for elem in pred if test_data[elem][1] == "T"])
    fn = len(curr["fn"])
    fp = len(curr["fp"])

    
    
    try:
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1score = 2 * precision * recall / (precision + recall)
        f2score = 5 * precision * recall / (4 * precision + recall)
        f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    except Exception as e:
        print (e)
        pass
            
    metrics = [precision, recall, f1score, f2score, f0_5score]
    failed.append(curr)
    results.append(metrics)

print ("Final Results:", np.mean(results, axis=0))

In [ ]:
# RotatE - complete

# Data Generation for RotatE embeddings from OpenKE

ontologies_in_alignment = list(set(flatten([l.split(".")[0].split("-") for l in os.listdir("reference-alignment/")])))

def generate(ontology):
    ont = Ontology(ontology)
    classes_dict = {elem : str(i) for i,elem in enumerate(ont.get_classes())} 
    props = ont.get_object_properties() + ont.get_data_properties() + ["subclass_of"]
    prop_dict = {elem : str(i) for i,elem in enumerate(props)} 
    triplets = ["\t".join((classes_dict[el[0]], classes_dict[el[1]], prop_dict[el[2]])) for el in ont.get_triples()]
    return classes_dict, prop_dict, triplets

all_elems = {}

for ont_name in ontologies_in_alignment:
    ont = "conference_ontologies/" + ont_name + ".owl"
    
    benchmark_dir = "OpenKE/benchmarks/" + ont_name
    if not os.path.isdir(benchmark_dir):
        os.mkdir(benchmark_dir)
    
    classes_dict, prop_dict, triplets = generate(ont)
    
    entity_str = str(len(classes_dict)) + "\n" + "\n".join(["\t".join(elem) for elem in classes_dict.items()])
    prop_str = str(len(prop_dict)) + "\n" + "\n".join(["\t".join(elem) for elem in prop_dict.items()])
    triplets_str = str(len(triplets)) + "\n" + "\n".join(triplets)
    print(classes_dict)
    print (ont_name, len(classes_dict.keys()), 
           len([el for el in classes_dict if el.split("#")[0] == "cmt"]))
    for key in classes_dict.keys():
        all_elems[ont_name+"#ENT@@"+key] = {"type": "entity"}
        
    for key in prop_dict.keys():
        all_elems[ont_name+"#PROP@@"+key] = {"type": "property"}
    
    open(benchmark_dir + "/entity2id.txt", "w+").write(entity_str)
    open(benchmark_dir + "/relation2id.txt", "w+").write(prop_str)
    open(benchmark_dir + "/train2id.txt", "w+").write(triplets_str)

# print (le)
openke_embeds = extractUSEEmbeddings([" ".join(parse(elem.split("@@")[1])) for elem in all_elems.keys()])
openke_embeds_dict = {key: {"type": all_elems[key]["type"], "embeds": openke_embeds[i]} for i,key in enumerate(all_elems)}


for ont_name in ontologies_in_alignment:
    benchmark_dir = "OpenKE/benchmarks/" + ont_name + "/"
    f = open(benchmark_dir + "embeddings.pkl", "wb")
    entity_embeds = {el.split("@@")[1]: np.concatenate((openke_embeds_dict[el]["embeds"], 
                                                      openke_embeds_dict[el]["embeds"]))
                     for el in openke_embeds_dict 
                     if el.split("#")[0] == ont_name and openke_embeds_dict[el]["type"] == "entity"}
    print (entity_embeds.keys())
    prop_embeds = {el.split("@@")[1]: openke_embeds_dict[el]["embeds"] for el in openke_embeds_dict 
     if el.split("#")[0] == ont_name and openke_embeds_dict[el]["type"] == "property"}
    
    pickle.dump([list(entity_embeds.values()),list(prop_embeds.values())], f)

# Processing results
ent_embeddings, rel_embeddings = {}, {}
for d in os.listdir("OpenKE/benchmarks/"):
    f = open("OpenKE/benchmarks/" + d + "/embeddings_final.pkl", "rb")
    ent_emb, rel_emb = pickle.load(f)
    entities = [d+"#"+l.rsplit("\t", 1)[0] for l in open("OpenKE/benchmarks/" + d + "/entity2id.txt", "r").read().split("\n")[1:]]
    relations = [d+"#"+l.rsplit("\t", 1)[0] for l in open("OpenKE/benchmarks/" + d + "/relation2id.txt", "r").read().split("\n")[1:]]
    if len(entities) != len(ent_emb):
        print ("Entity legnth mismatch")
        break
    if len(relations) != len(rel_emb):
        print ("relation legnth mismatch")
        break
    ent_embeddings = {**ent_embeddings, **dict(zip(entities, ent_emb.numpy()))}
    rel_embeddings = {**rel_embeddings, **dict(zip(relations, rel_emb.numpy()))}
    


In [ ]:

def get_property_features(ont_name, triple):

    weight = np.sum([get_tfidf_score(word, triple[-1]) for word in parse(triple[-1])])
    feats = []
    feats.append(weight*embeddings[ont_name + "#" + triple[-1]]) # Property name
    feats.append([embeddings[ont_name + "#" + el] for el in triple[0].split("###")]) # Domain 
    feats.append([embeddings[ont_name + "#" + el] for el in triple[1].split("###")]) # Range
    return feats
    
def get_entity_features(ont_name, entity):
    feats = []
    weight = np.sum([get_tfidf_score(word, entity) for word in parse(entity)])
    feats.append(weight * embeddings[ont_name + "#" + entity]) # Entity name
#     feats.append(np.sum([get_tfidf_score(word, entity) * embeddings[word] for word in parse(entity)]))
    return feats

for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    
    triples = ont.get_triples(union_flag=1, subclass_of=False)
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()

    for triple in triples:
        features_dict[triple[-1]] = {"type": "triple", "features": get_property_features(ont_name, triple)}
    for entity in entities:
        features_dict[entity] = {"type": "entity", "features": get_entity_features(ont_name, entity)}
    for prop in props:
        if prop not in features_dict:
            features_dict[prop] = {"type": "property", "features": get_entity_features(ont_name, prop)}

            


In [ ]:
# Dot product similarity calculation

def calc_sim_score(mapping):
    features = embedify(mapping)
    if features[0]["type"] != "triple" and features[1]["type"] != "triple":
        return cos_sim(*tuple([el["features"][0] for el in features]))
    elif features[0]["type"] == "triple"  and features[1]["type"] == "triple":
        sim_score = 0
        names, domains, ranges = [], [], []
        
        for elem in [el["features"] for el in features]:
            names.append(elem[0])
            domains.append(elem[1])
            ranges.append(elem[2])
        name_score = cos_sim(names[0], names[1])
#         domain_score = np.mean([cos_sim(*elem) for elem in itertools.product(domains[0], domains[1])])
#         ranges_score = np.mean([cos_sim(*elem) for elem in itertools.product(ranges[0], ranges[1])])
#         return ((name_score + domain_score + ranges_score)/(len(domains[0]) + len(ranges[0]) +  ))
#         return (name_score + domain_score + ranges_score)/3
        return name_score
    return cos_sim(*tuple([el["features"][0] for el in features])) 
        
        

def embedify(mapping):
    removed_hash = tuple([elem.split("#")[1] for elem in mapping])
    return (features_dict[removed_hash[0]], features_dict[removed_hash[1]])

gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in reference_alignments]

data = {}
for mapping in all_mappings:
    if mapping in gt_mappings:
        data[(mapping[0], mapping[1])] = (calc_sim_score(mapping), "T")
    else:
        data[(mapping[0], mapping[1])] = (calc_sim_score(mapping), "F")


In [ ]:

inp = [" ".join(parse(word.split("#")[1])) for word in extracted_elems]
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\S+")
X = vectorizer.fit_transform(inp)
word2idx_tfidf = {word: i for (i, word)  in enumerate(vectorizer.get_feature_names())}
entity2idx_tfidf = {word.split("#")[1]: i for (i, word)  in enumerate(extracted_elems)}



In [ ]:
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import tensorflow_hub as hub

f = open("data.pkl", "rb")
data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, all_ont_pairs = pickle.load(f)

def create_embeddings_file(embeds, name):
    global extracted_elems, data, gt_mappings, all_ont_pairs, emb_indexer, emb_indexer_inv
    embedding_dim = embeds.shape[1]
    embeds = np.array([np.zeros(embedding_dim)] + list(embeds))
    embeddings = dict(zip(extracted_elems, embeds))

    emb_vals = list(embeddings.values())


    f = open("data_" + name + ".pkl", "wb")
    pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, all_ont_pairs], f)

def elmo_vectors(x):
    embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))


elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
emb = elmo_vectors(inp)
create_embeddings_file(emb, "elmov2")

from sentence_transformers import SentenceTransformer
# roberta_large = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
# bert_large = SentenceTransformer("bert-large-nli-stsb-mean-tokens")
roberta_large_nli = SentenceTransformer('roberta-large-nli-mean-tokens')
bert_large_nli = SentenceTransformer('bert-large-nli-mean-tokens')
distilbert_large_nli = SentenceTransformer('distilbert-base-nli-mean-tokens')

distilbert_large = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')